## knock 085 郵便番号からジオコードに変換する

In [1]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [2]:
df_customer = pl.read_csv("../docker/work/data/customer.csv")
display(df_customer.head())

df_geocode = pl.read_csv("../docker/work/data/geocode.csv")
display(df_geocode.head())

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071…","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂******…","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172…","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木*****…","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001…","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**…","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145…","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷*****…","""S13001""",20170605,"""6-20090929-2"""


postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
str,str,str,str,str,str,str,f64,f64
"""060-0000""","""北海道""","""札幌市中央区""",null,null,null,"""北海道札幌市中央区""",141.34103,43.05513
"""064-0941""","""北海道""","""札幌市中央区""","""旭ケ丘""",null,null,"""北海道札幌市中央区旭ケ丘""",141.31972,43.04223
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""１丁目""","""北海道札幌市中央区大通西１丁…",141.35637,43.06102
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""２丁目""","""北海道札幌市中央区大通西２丁…",141.35445,43.0608
"""060-0042""","""北海道""","""札幌市中央区""","""大通西""",null,"""３丁目""","""北海道札幌市中央区大通西３丁…",141.35275,43.06086


### ノック

In [3]:
df_knock = (
    df_customer
    # 郵便番号ごとの経度(longitude)・緯度(latitude)の平均値を加える
    .join(
        other =
        df_geocode
        .group_by(by = "postal_cd")
        .agg([
            pl.mean("longitude"),
            pl.mean("latitude")
        ]),
        on = "postal_cd",
        how = "left"        
    )
)

# 先頭10行を表示する
df_knock.head(n = 10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
str,str,i64,str,str,i64,str,str,str,i64,str,f64,f64
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0""",139.31779,35.41358
"""CS037613000071…","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂******…","""S13037""",20150414,"""0-00000000-0""",139.83502,35.67193
"""CS031415000172…","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木*****…","""S13031""",20150529,"""D-20100325-C""",139.68965,35.67374
"""CS028811000001…","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**…","""S14028""",20160115,"""0-00000000-0""",139.4836,35.39125
"""CS001215000145…","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷*****…","""S13001""",20170605,"""6-20090929-2""",139.70775,35.54084
"""CS020401000016…","""宮下 達士""",0,"""男性""","""1974-09-15""",44,"""174-0065""","""東京都板橋区若木******…","""S13020""",20150225,"""0-00000000-0""",139.67245,35.77073
"""CS015414000103…","""奥野 陽子""",1,"""女性""","""1977-08-09""",41,"""136-0073""","""東京都江東区北砂******…","""S13015""",20150722,"""B-20100609-B""",139.83601,35.67818
"""CS029403000008…","""釈 人志""",0,"""男性""","""1973-08-17""",45,"""279-0003""","""千葉県浦安市海楽******…","""S12029""",20150515,"""0-00000000-0""",139.90469,35.65422
"""CS015804000004…","""松谷 米蔵""",0,"""男性""","""1931-05-02""",87,"""136-0073""","""東京都江東区北砂******…","""S13015""",20150607,"""0-00000000-0""",139.83601,35.67818


### 検算

In [5]:
df_knock.filter(
    (pl.col("customer_id") == "CS021313000114") |
    (pl.col("customer_id") == "CS021303000023") |
    (pl.col("customer_id") == "CS021303000007") |
    (pl.col("customer_id") == "CS021313000183") |
    (pl.col("customer_id") == "CS021314000098") |

    (pl.col("customer_id") == "CS021314000093") |
    (pl.col("customer_id") == "CS021413000049") |
    (pl.col("customer_id") == "CS021313000025") |
    (pl.col("customer_id") == "CS021413000022") |
    (pl.col("customer_id") == "CS021413000094")
)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
str,str,i64,str,str,i64,str,str,str,i64,str,f64,f64
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0""",139.31779,35.41358
"""CS021313000025…","""砂川 あさみ""",1,"""女性""","""1981-10-05""",37,"""259-1131""","""神奈川県伊勢原市伊勢原***…","""S14021""",20150326,"""0-00000000-0""",139.31374,35.39748
"""CS021413000022…","""布施 花""",1,"""女性""","""1972-05-18""",46,"""259-1131""","""神奈川県伊勢原市伊勢原***…","""S14021""",20150225,"""0-00000000-0""",139.31374,35.39748
"""CS021413000094…","""宇佐美 メイサ""",1,"""女性""","""1970-05-18""",48,"""259-1131""","""神奈川県伊勢原市伊勢原***…","""S14021""",20150422,"""0-00000000-0""",139.31374,35.39748
"""CS021303000023…","""堀 一徳""",0,"""男性""","""1980-04-05""",38,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20160411,"""0-00000000-0""",139.31779,35.41358
"""CS021303000007…","""石倉 俊二""",0,"""男性""","""1987-07-04""",31,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150707,"""0-00000000-0""",139.31779,35.41358
"""CS021313000183…","""草野 未來""",1,"""女性""","""1986-12-21""",32,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20170611,"""0-00000000-0""",139.31779,35.41358
"""CS021314000098…","""筒井 れいな""",1,"""女性""","""1985-09-21""",33,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20160901,"""0-00000000-0""",139.31779,35.41358
"""CS021314000093…","""江藤 美佐""",1,"""女性""","""1986-06-03""",32,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20151015,"""0-00000000-0""",139.31779,35.41358
